# Transform Khách Hàng: Silver → Gold

## Mục tiêu
Chuyển đổi dữ liệu khách hàng từ tầng Silver sang tầng Gold với các cải tiến:
- Chuẩn hóa dữ liệu (giới tính, số điện thoại)
- Tính toán RFM (Recency, Frequency, Monetary)
- Phân loại khách hàng VIP
- Enrich thông tin địa lý

## Quy trình thực hiện
1. **Kết nối database** - Thiết lập connection đến Bronze, Silver, Gold
2. **Load dữ liệu bổ trợ** - Lấy danh sách tỉnh thành từ Bronze
3. **Load dữ liệu chính** - Lấy thông tin khách hàng từ Silver
4. **Khám phá dữ liệu** - Phân tích cấu trúc và chất lượng
5. **Transform dữ liệu** - Áp dụng các quy tắc chuẩn hóa và tính toán
6. **Kiểm tra kết quả** - Validate dữ liệu sau transform
7. **Load vào Gold** - Lưu trữ dữ liệu đã chuẩn hóa


In [5]:
# ==========================
# Import thư viện và thiết lập kết nối database
# ==========================

import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

print("Đang khởi tạo kết nối database...")

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS") 
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")
DB_SILVER = os.getenv("DB_SILVER")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo connection string cho từng tầng
bronze_conn = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}"
silver_conn = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}"
gold_conn = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}"

# Tạo engine cho từng tầng
engine_bronze = create_engine(bronze_conn)
engine_silver = create_engine(silver_conn)
engine_gold = create_engine(gold_conn)

print("✅ Kết nối database thành công!")
print(f"   - Bronze: {DB_BRONZE}")
print(f"   - Silver: {DB_SILVER}")
print(f"   - Gold: {DB_GOLD}")


Đang khởi tạo kết nối database...
✅ Kết nối database thành công!
   - Bronze: winner_bronze
   - Silver: winner_silver
   - Gold: winner_gold


## Bước 1: Kết nối Database

Thiết lập kết nối đến cả 3 tầng database:
- **Bronze**: Lấy dữ liệu bổ trợ (tỉnh thành)
- **Silver**: Lấy dữ liệu khách hàng đã làm sạch
- **Gold**: Lưu trữ dữ liệu sau transform

Sử dụng SQLAlchemy với PyMySQL driver để kết nối MySQL.


In [6]:
# ==========================
# Load dữ liệu tỉnh thành từ Bronze
# ==========================

print("Đang load dữ liệu tỉnh thành...")

# Query lấy tất cả tỉnh thành
query_provinces = """
SELECT 
    province_id,
    province_name
FROM province_raw
ORDER BY province_name
"""

# Thực hiện query
df_provinces = pd.read_sql(query_provinces, engine_bronze)

print(f"✅ Load thành công {df_provinces.shape[0]} tỉnh thành")
print(f"   - Cấu trúc: {df_provinces.shape[1]} cột")
print(f"   - Cột: {list(df_provinces.columns)}")

# Hiển thị một số tỉnh thành mẫu
print("\nMột số tỉnh thành mẫu:")
print(df_provinces.head(10).to_string(index=False))


Đang load dữ liệu tỉnh thành...
✅ Load thành công 63 tỉnh thành
   - Cấu trúc: 2 cột
   - Cột: ['province_id', 'province_name']

Một số tỉnh thành mẫu:
province_id   province_name
        805        An Giang
        717 Bà Rịa-Vũng Tàu
        221       Bắc Giang
        207         Bắc Kạn
        821        Bạc Liêu
        106        Bắc Ninh
        811         Bến Tre
        507       Bình Định
        711      Bình Dương
        707      Bình Phước


## Bước 2: Load Dữ Liệu Bổ Trợ

Load bảng `province_raw` từ Bronze để có thông tin tỉnh thành:
- **Mục đích**: Join với khách hàng để có tên tỉnh thay vì chỉ ID
- **Cấu trúc**: province_id, province_name
- **Số lượng**: 63 tỉnh thành Việt Nam

Dữ liệu này sẽ được sử dụng để enrich thông tin địa lý của khách hàng.


In [7]:
# ==========================
# Load dữ liệu khách hàng từ Silver
# ==========================

print("Đang load dữ liệu khách hàng...")

# Query lấy tất cả thông tin khách hàng
query_customers = """
SELECT 
    customer_id,
    shop_id,
    name,
    gender,
    phone,
    fb_id,
    order_count,
    succeed_order_count,
    returned_order_count,
    purchased_amount,
    reward_point,
    current_debts,
    count_referrals,
    inserted_at,
    updated_at,
    last_order_at,
    is_block,
    is_discount_by_level,
    active_levera_pay,
    province_id,
    full_address,
    referral_code,
    order_sources
FROM dim_customers
ORDER BY inserted_at DESC
"""

# Thực hiện query
df_customers = pd.read_sql(query_customers, engine_silver)

print(f"✅ Load thành công {df_customers.shape[0]:,} khách hàng")
print(f"   - Cấu trúc: {df_customers.shape[1]} cột")
print(f"   - Cột: {list(df_customers.columns)}")

# Hiển thị thông tin cơ bản
print(f"\nThông tin cơ bản:")
print(f"   - Khách hàng đầu tiên: {df_customers['inserted_at'].max()}")
print(f"   - Khách hàng cuối cùng: {df_customers['inserted_at'].min()}")
print(f"   - Tổng doanh thu: {df_customers['purchased_amount'].sum():,} VND")

# Hiển thị một số khách hàng mẫu
print("\nMột số khách hàng mẫu:")
sample_cols = ['customer_id', 'name', 'gender', 'order_count', 'purchased_amount']
print(df_customers[sample_cols].head(5).to_string(index=False))


Đang load dữ liệu khách hàng...
✅ Load thành công 36,090 khách hàng
   - Cấu trúc: 23 cột
   - Cột: ['customer_id', 'shop_id', 'name', 'gender', 'phone', 'fb_id', 'order_count', 'succeed_order_count', 'returned_order_count', 'purchased_amount', 'reward_point', 'current_debts', 'count_referrals', 'inserted_at', 'updated_at', 'last_order_at', 'is_block', 'is_discount_by_level', 'active_levera_pay', 'province_id', 'full_address', 'referral_code', 'order_sources']

Thông tin cơ bản:
   - Khách hàng đầu tiên: 2025-08-16 02:19:59
   - Khách hàng cuối cùng: 2021-12-30 03:13:14
   - Tổng doanh thu: 8,018,929,990 VND

Một số khách hàng mẫu:
                         customer_id        name gender  order_count  purchased_amount
c7a583d6-392d-4308-a02b-67c001ee03db   Thinh Bui      M            1                 0
590ba7de-e4fe-43b3-8013-27fc69937edf Truong Minh      M            1                 0
a9e62388-57c2-4834-a87c-7c1025aea92d Quach Quach      M            1                 0
002810c4-5e9

## Bước 3: Load Dữ Liệu Khách Hàng

Load bảng `dim_customers` từ Silver:
- **Nguồn**: Dữ liệu đã được làm sạch và chuẩn hóa ở tầng Silver
- **Kích thước**: 36,090 khách hàng với 23 cột
- **Các thông tin chính**: 
  - Thông tin cá nhân (tên, giới tính, SĐT, địa chỉ)
  - Thông tin giao dịch (số đơn hàng, doanh thu, điểm thưởng)
  - Trạng thái khách hàng (block, discount level)

Dữ liệu này sẽ được transform để tạo ra dimension table cho phân tích.


In [8]:
# ==========================
# Phân tích chất lượng và cấu trúc dữ liệu
# ==========================

print("Đang phân tích chất lượng dữ liệu...")

# 1. Thông tin cơ bản về dataset
print("="*60)
print("THÔNG TIN CƠ BẢN VỀ DATASET")
print("="*60)
print(f"Kích thước: {df_customers.shape[0]:,} dòng × {df_customers.shape[1]} cột")
print(f"Memory usage: {df_customers.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# 2. Phân tích missing values
print("\n" + "="*60)
print("PHÂN TÍCH MISSING VALUES")
print("="*60)
missing_data = df_customers.isnull().sum()
missing_percent = (missing_data / len(df_customers)) * 100

missing_df = pd.DataFrame({
    'Cột': missing_data.index,
    'Số_Lượng_Missing': missing_data.values,
    'Tỷ_Lệ_%': missing_percent.values.round(2)
}).sort_values('Tỷ_Lệ_%', ascending=False)

print(missing_df[missing_df['Số_Lượng_Missing'] > 0].to_string(index=False))

# 3. Thống kê mô tả cho các biến số
print("\n" + "="*60)
print("THỐNG KÊ MÔ TẢ - CÁC BIẾN SỐ")
print("="*60)
numeric_cols = ['order_count', 'succeed_order_count', 'returned_order_count', 
                'purchased_amount', 'reward_point', 'current_debts', 'count_referrals']
print(df_customers[numeric_cols].describe().round(2))

# 4. Phân tích biến phân loại
print("\n" + "="*60)
print("PHÂN TÍCH BIẾN PHÂN LOẠI")
print("="*60)
categorical_cols = ['gender', 'is_block', 'is_discount_by_level', 'active_levera_pay']

for col in categorical_cols:
    if col in df_customers.columns:
        print(f"\n{col.upper()}:")
        value_counts = df_customers[col].value_counts()
        for value, count in value_counts.items():
            percent = (count / len(df_customers)) * 100
            print(f"  {value}: {count:,} ({percent:.1f}%)")

Đang phân tích chất lượng dữ liệu...
THÔNG TIN CƠ BẢN VỀ DATASET
Kích thước: 36,090 dòng × 23 cột
Memory usage: 27.80 MB

PHÂN TÍCH MISSING VALUES
          Cột  Số_Lượng_Missing  Tỷ_Lệ_%
last_order_at              9414    26.08
       gender              5361    14.85
  province_id              3641    10.09
 full_address              3631    10.06
        fb_id               135     0.37
        phone                90     0.25
order_sources                30     0.08
         name                 7     0.02

THỐNG KÊ MÔ TẢ - CÁC BIẾN SỐ
       order_count  succeed_order_count  returned_order_count  \
count     36090.00             36090.00              36090.00   
mean          1.12                 0.49                  0.38   
std           0.39                 0.54                  0.51   
min           0.00                 0.00                  0.00   
25%           1.00                 0.00                  0.00   
50%           1.00                 0.00                  0.00   

## Bước 4: Khám Phá Dữ Liệu

### Phân tích cấu trúc dữ liệu
- **Kiểm tra info()**: Xem kiểu dữ liệu, số lượng null values
- **Thống kê mô tả**: Phân tích phân bố các biến số
- **Phân tích giới tính**: Xem tỷ lệ nam/nữ/khác

### Insights từ dữ liệu
- **Chất lượng dữ liệu**: Một số cột có null values (gender: 14.9%, province: 10.1%)
- **Phân bố giới tính**: Nữ chiếm đa số (54%), Nam (31.1%), Khác (14.9%)
- **Hành vi mua hàng**: Trung bình 1.1 đơn hàng/khách, doanh thu TB 222k VND

Những thông tin này giúp hiểu đặc điểm khách hàng trước khi transform.


In [9]:
# ==========================
# Phân tích chi tiết về giới tính khách hàng
# ==========================

print("Phân tích phân bố giới tính khách hàng:")
print("-" * 50)

gender_analysis = df_customers['gender'].value_counts()
total_customers = len(df_customers)

print(f"Tổng số khách hàng: {total_customers:,}")
print("\nPhân bố theo giới tính:")
for gender, count in gender_analysis.items():
    percentage = (count / total_customers) * 100
    print(f"  {gender}: {count:,} khách hàng ({percentage:.1f}%)")

# Phân tích giới tính theo doanh thu
print("\nPhân tích doanh thu theo giới tính:")
gender_revenue = df_customers.groupby('gender')['purchased_amount'].agg(['count', 'sum', 'mean']).round(0)
gender_revenue.columns = ['Số_Khách', 'Tổng_Doanh_Thu', 'Doanh_Thu_TB']
print(gender_revenue.to_string())

Phân tích phân bố giới tính khách hàng:
--------------------------------------------------
Tổng số khách hàng: 36,090

Phân bố theo giới tính:
  F: 19,486 khách hàng (54.0%)
  M: 11,239 khách hàng (31.1%)
  O: 4 khách hàng (0.0%)

Phân tích doanh thu theo giới tính:
        Số_Khách  Tổng_Doanh_Thu  Doanh_Thu_TB
gender                                        
F          19486      4125492950      211716.0
M          11239      2962436040      263585.0
O              4          609000      152250.0


## Bước 5: Transform Dữ Liệu

### Logic transform trong hàm `transform_customers()`

**B1. Chuẩn hóa giới tính**
- M → Nam, F → Nữ, O → Khác
- Xử lý missing values

**B2. Chuẩn hóa số điện thoại**
- Loại bỏ ký tự đặc biệt
- Chỉ giữ 10 số cuối

**B3. Phân loại VIP**
- Khách hàng có purchased_amount >= 500,000 VND → VIP
- Ngưỡng VIP có thể điều chỉnh theo business requirement

**B4. Tính RFM cơ bản**
- **Recency**: Số ngày từ lần mua cuối đến hiện tại
- **Frequency**: Số đơn hàng (order_count)
- **Monetary**: Tổng giá trị mua hàng (purchased_amount)

**B5. Enrich địa lý**
- Join với bảng tỉnh thành để có tên tỉnh

**B6. Chọn cột quan trọng**
- Giữ lại các cột cần thiết cho phân tích
- Loại bỏ các cột không sử dụng


In [10]:
# ==========================
# Định nghĩa hàm transform dữ liệu khách hàng
# ==========================

def transform_customers(df_customers, df_provinces=None):
    """
    Transform dữ liệu khách hàng từ Silver sang Gold
    
    Args:
        df_customers (DataFrame): Dữ liệu khách hàng từ Silver
        df_provinces (DataFrame): Dữ liệu tỉnh thành từ Bronze
    
    Returns:
        DataFrame: Dữ liệu khách hàng đã được transform cho Gold
    """
    print("Bắt đầu transform dữ liệu khách hàng...")
    
    # Tạo bản copy để tránh thay đổi dữ liệu gốc
    df = df_customers.copy()
    original_count = len(df)
    
    # B1. Chuẩn hóa giới tính
    print("  - Chuẩn hóa giới tính...")
    gender_mapping = {
        'M': 'Nam',
        'F': 'Nữ', 
        'O': 'Khác'
    }
    df['gender'] = df['gender'].map(gender_mapping).fillna('Khác')
    
    # Thống kê sau chuẩn hóa giới tính
    gender_stats = df['gender'].value_counts()
    print(f"    + Nam: {gender_stats.get('Nam', 0):,} khách")
    print(f"    + Nữ: {gender_stats.get('Nữ', 0):,} khách")
    print(f"    + Khác: {gender_stats.get('Khác', 0):,} khách")

    # B2. Chuẩn hóa số điện thoại
    print("  - Chuẩn hóa số điện thoại...")
    df['phone'] = df['phone'].astype(str).str.replace(r'\D', '', regex=True).str[-10:]
    valid_phones = df['phone'].str.len() == 10
    print(f"    + Số điện thoại hợp lệ: {valid_phones.sum():,}")

    # B3. Phân loại khách hàng VIP
    print("  - Phân loại khách hàng VIP...")
    vip_threshold = 500000  # 500k VND
    df['is_vip'] = (df['purchased_amount'] >= vip_threshold).astype(int)
    vip_count = df['is_vip'].sum()
    print(f"    + Ngưỡng VIP: {vip_threshold:,} VND")
    print(f"    + Số khách VIP: {vip_count:,} ({vip_count/len(df)*100:.1f}%)")

    # B4. Tính toán RFM Analysis
    print("  - Tính toán RFM Analysis...")
    today = pd.to_datetime("today")
    
    # Recency: Số ngày từ lần mua cuối
    df['recency_days'] = (today - pd.to_datetime(df['last_order_at'])).dt.days
    df['recency_days'] = df['recency_days'].fillna(-1)  # -1 cho khách chưa mua
    
    # Frequency: Số đơn hàng
    df['frequency'] = df['order_count']
    
    # Monetary: Tổng giá trị mua hàng
    df['monetary'] = df['purchased_amount']
    
    print(f"    + Recency - Min: {df['recency_days'].min()}, Max: {df['recency_days'].max()}")
    print(f"    + Frequency - Min: {df['frequency'].min()}, Max: {df['frequency'].max()}")
    print(f"    + Monetary - Min: {df['monetary'].min():,}, Max: {df['monetary'].max():,}")

    # B5. Enrich thông tin địa lý
    if df_provinces is not None:
        print("  - Enrich thông tin địa lý...")
        # Merge với bảng tỉnh thành
        df = df.merge(
            df_provinces[['province_id', 'province_name']], 
            on='province_id', 
            how='left'
        )
        province_coverage = df['province_name'].notna().sum()
        print(f"    + Khách hàng có thông tin tỉnh: {province_coverage:,} ({province_coverage/len(df)*100:.1f}%)")

    # B6. Chọn và sắp xếp các cột quan trọng
    print("  - Chọn lọc cột quan trọng...")
    gold_columns = [
        'customer_id', 'shop_id', 'name', 'gender', 'phone',
        'order_count', 'succeed_order_count', 'returned_order_count',
        'purchased_amount', 'last_order_at',
        'recency_days', 'frequency', 'monetary', 'is_vip',
        'province_id', 'province_name'
    ]
    
    # Chỉ giữ lại các cột có trong dataset
    available_columns = [col for col in gold_columns if col in df.columns]
    df_gold = df[available_columns].copy()
    
    print(f"    + Số cột ban đầu: {len(df.columns)}")
    print(f"    + Số cột sau transform: {len(df_gold.columns)}")
    print(f"    + Số khách hàng: {len(df_gold):,} (không thay đổi)")
    
    print("✅ Transform hoàn tất!")
    return df_gold


## Bước 6: Áp Dụng Transform

Thực hiện transform dữ liệu khách hàng:
- **Input**: df_customers (Silver) + df_provinces (Bronze)
- **Output**: df_gold_customers với 16 cột đã được chuẩn hóa
- **Kết quả**: 36,090 khách hàng với đầy đủ thông tin RFM và phân loại VIP

Dữ liệu sau transform sẽ sẵn sàng cho việc phân tích và tạo dashboard.


In [11]:
# ==========================
# Áp dụng transform cho dữ liệu khách hàng
# ==========================

print("Thực hiện transform dữ liệu khách hàng...")
print("="*60)

# Áp dụng hàm transform
df_gold_customers = transform_customers(df_customers, df_provinces)

# Hiển thị kết quả
print("\n" + "="*60)
print("KẾT QUẢ SAU TRANSFORM")
print("="*60)

print(f"Kích thước dataset: {df_gold_customers.shape[0]:,} dòng × {df_gold_customers.shape[1]} cột")
print(f"Các cột trong dataset:")
for i, col in enumerate(df_gold_customers.columns, 1):
    print(f"  {i:2d}. {col}")

# Hiển thị sample data
print(f"\nMẫu dữ liệu sau transform:")
sample_cols = ['customer_id', 'name', 'gender', 'phone', 'is_vip', 'purchased_amount', 'province_name']
available_sample_cols = [col for col in sample_cols if col in df_gold_customers.columns]
print(df_gold_customers[available_sample_cols].head(10).to_string(index=False))

Thực hiện transform dữ liệu khách hàng...
Bắt đầu transform dữ liệu khách hàng...
  - Chuẩn hóa giới tính...
    + Nam: 11,239 khách


    + Nữ: 19,486 khách
    + Khác: 5,365 khách
  - Chuẩn hóa số điện thoại...
    + Số điện thoại hợp lệ: 35,999
  - Phân loại khách hàng VIP...
    + Ngưỡng VIP: 500,000 VND
    + Số khách VIP: 7,323 (20.3%)
  - Tính toán RFM Analysis...
    + Recency - Min: -1.0, Max: 1375.0
    + Frequency - Min: 0, Max: 11
    + Monetary - Min: -1,788,000, Max: 6,199,000
  - Enrich thông tin địa lý...
    + Khách hàng có thông tin tỉnh: 32,449 (89.9%)
  - Chọn lọc cột quan trọng...
    + Số cột ban đầu: 28
    + Số cột sau transform: 16
    + Số khách hàng: 36,090 (không thay đổi)
✅ Transform hoàn tất!

KẾT QUẢ SAU TRANSFORM
Kích thước dataset: 36,090 dòng × 16 cột
Các cột trong dataset:
   1. customer_id
   2. shop_id
   3. name
   4. gender
   5. phone
   6. order_count
   7. succeed_order_count
   8. returned_order_count
   9. purchased_amount
  10. last_order_at
  11. recency_days
  12. frequency
  13. monetary
  14. is_vip
  15. province_id
  16. province_name

Mẫu dữ liệu sau transform:
    

## Bước 7: Kiểm Tra Kết Quả

### Validation dữ liệu sau transform
- **Phân loại VIP**: 7,323 khách VIP (20.3%) vs 28,767 khách thường (79.7%)
- **Ngưỡng VIP**: 500,000 VND là ngưỡng hợp lý cho phân khúc khách hàng cao cấp
- **Chất lượng dữ liệu**: Tất cả 36,090 khách hàng đều được xử lý thành công

### Kết quả đạt được
- **RFM Analysis**: Có đầy đủ thông tin Recency, Frequency, Monetary
- **Geographic Enrichment**: Có tên tỉnh thành cho phân tích địa lý
- **Data Quality**: Chuẩn hóa giới tính, số điện thoại
- **Business Logic**: Phân loại VIP theo ngưỡng doanh thu

Dữ liệu đã sẵn sàng để load vào Gold database.


In [12]:
# ==========================
# Validation kết quả transform
# ==========================

print("Kiểm tra kết quả transform...")
print("="*60)

# 1. Kiểm tra phân loại VIP
print("1. PHÂN LOẠI KHÁCH HÀNG VIP:")
print("-" * 30)
vip_counts = df_gold_customers['is_vip'].value_counts()
total_customers = len(df_gold_customers)

for vip_status, count in vip_counts.items():
    status_name = "VIP" if vip_status == 1 else "Thường"
    percentage = (count / total_customers) * 100
    print(f"   {status_name}: {count:,} khách hàng ({percentage:.1f}%)")

# 2. Kiểm tra RFM Analysis
print(f"\n2. RFM ANALYSIS:")
print("-" * 30)
print(f"   Recency (ngày): Min={df_gold_customers['recency_days'].min()}, Max={df_gold_customers['recency_days'].max()}")
print(f"   Frequency (đơn hàng): Min={df_gold_customers['frequency'].min()}, Max={df_gold_customers['frequency'].max()}")
print(f"   Monetary (VND): Min={df_gold_customers['monetary'].min():,}, Max={df_gold_customers['monetary'].max():,}")

# 3. Kiểm tra chất lượng dữ liệu
print(f"\n3. CHẤT LƯỢNG DỮ LIỆU:")
print("-" * 30)
missing_data = df_gold_customers.isnull().sum()
if missing_data.sum() > 0:
    print("   Các cột có missing values:")
    for col, count in missing_data.items():
        if count > 0:
            percentage = (count / len(df_gold_customers)) * 100
            print(f"     - {col}: {count:,} ({percentage:.1f}%)")
else:
    print("   ✅ Không có missing values")

# 4. Kiểm tra duplicate
print(f"\n4. KIỂM TRA DUPLICATE:")
print("-" * 30)
duplicate_count = df_gold_customers['customer_id'].duplicated().sum()
if duplicate_count > 0:
    print(f"   ⚠️ Có {duplicate_count} customer_id trùng lặp")
else:
    print("   ✅ Không có customer_id trùng lặp")

print(f"\n✅ Validation hoàn tất!")

Kiểm tra kết quả transform...
1. PHÂN LOẠI KHÁCH HÀNG VIP:
------------------------------
   Thường: 28,767 khách hàng (79.7%)
   VIP: 7,323 khách hàng (20.3%)

2. RFM ANALYSIS:
------------------------------
   Recency (ngày): Min=-1.0, Max=1375.0
   Frequency (đơn hàng): Min=0, Max=11
   Monetary (VND): Min=-1,788,000, Max=6,199,000

3. CHẤT LƯỢNG DỮ LIỆU:
------------------------------
   Các cột có missing values:
     - name: 7 (0.0%)
     - last_order_at: 9,414 (26.1%)
     - province_id: 3,641 (10.1%)
     - province_name: 3,641 (10.1%)

4. KIỂM TRA DUPLICATE:
------------------------------
   ✅ Không có customer_id trùng lặp

✅ Validation hoàn tất!


## Bước 8: Load Vào Gold Database

### Lưu trữ dữ liệu đã transform
- **Target**: Bảng `gold_dim_customers` trong Gold schema
- **Method**: Sử dụng pandas.to_sql() với SQLAlchemy engine
- **Mode**: replace (ghi đè nếu bảng đã tồn tại)
- **Kết quả**: 36,090 records được load thành công

### Cấu trúc bảng Gold
Bảng `gold_dim_customers` chứa:
- **Thông tin cơ bản**: customer_id, shop_id, name, gender, phone
- **Hành vi mua hàng**: order_count, succeed_order_count, purchased_amount
- **RFM Metrics**: recency_days, frequency, monetary
- **Phân loại**: is_vip
- **Địa lý**: province_id, province_name

Dữ liệu này sẽ được sử dụng cho các phân tích và dashboard trong tương lai.


In [13]:
# # ==========================
# # Load dữ liệu vào Gold Database
# # ==========================

# print("Đang load dữ liệu vào Gold Database...")
# print("="*60)

# # Thông tin trước khi load
# print(f"Dataset cần load:")
# print(f"  - Số dòng: {len(df_gold_customers):,}")
# print(f"  - Số cột: {len(df_gold_customers.columns)}")
# print(f"  - Tên bảng: gold_dim_customers")
# print(f"  - Schema: {DB_GOLD}")

# # Load dữ liệu vào database
# try:
#     df_gold_customers.to_sql(
#         "gold_dim_customers",
#         engine_gold,
#         if_exists="replace",
#         index=False,
#         chunksize=1000,  # Load theo batch để tối ưu memory
#         method='multi'   # Sử dụng multi-insert để tăng tốc
#     )
    
#     print(f"\n✅ Load thành công!")
#     print(f"  - Bảng: gold_dim_customers")
#     print(f"  - Records: {len(df_gold_customers):,}")
#     print(f"  - Schema: {DB_GOLD}")
    
#     # Verify dữ liệu đã được load
#     print(f"\nĐang verify dữ liệu...")
#     verify_query = "SELECT COUNT(*) as total_records FROM gold_dim_customers"
#     result = pd.read_sql(verify_query, engine_gold)
#     loaded_count = result['total_records'].iloc[0]
    
#     if loaded_count == len(df_gold_customers):
#         print(f"✅ Verify thành công: {loaded_count:,} records")
#     else:
#         print(f"⚠️ Warning: Expected {len(df_gold_customers):,}, but loaded {loaded_count:,}")
        
# except Exception as e:
#     print(f"❌ Lỗi khi load dữ liệu: {str(e)}")
#     raise

# print(f"\n🎉 Hoàn tất transform khách hàng từ Silver sang Gold!")